<a href="https://colab.research.google.com/github/tsangrebecca/BloomTech/blob/main/Sprint13/M3_DS_413_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a [kaggle competition](https://www.kaggle.com/c/whiskey-201911/) We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

We are going to run increasingly sophisticated classification models on our whisky reviews in parts 1, 2, and 3. For each of parts 1, 2, and 3, submit your best model's results to the Kaggle competition to measure `generalization accuracy` -- i.e. how well the model performs on new data.

##1. Classifier based on TfIdf vectorization of reviews

### Follow Along

1. Join the Kaggle Competition
2. Download the data
3. Train and hyperparameter tune a model using an sklearn pipeline

### 1.0 Setup

#### 1.0.1 Get spacy and restart runtime

In [1]:
#YOUR CODE HERE
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### 1.0.2 import necessary packages, load spacy

In [51]:
import pandas as pd
import re

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy

Load `spacy`

In [2]:
#YOUR CODE HERE
nlp = spacy.load("en_core_web_md") # disable=['parser', 'tagger'])

#### 1.0.3 Load Kaggle Whisky Competition Data
The goal is to predict the rating from the review text

In [36]:
# !!!!! You may need to change the path !!!!!
# You can download these datasets from the Kaggle in-class
# competition for your cohort.

# Copy and past the train and test data files and paste to Colab directory in the content folder

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [37]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [38]:
test.shape

(288, 2)

### 1.1 Clean Text

In [39]:
def clean_doc(text):
  # COMPLETE THE CODE IN THIS CELL
  # remove new line characters
  text = text.replace('\\n', ' ')
  # remove numbers from the text
  text = re.sub('[^a-zA-Z]', ' ', text) # keep^ letters
  # remove multiple white spaces
  text = re.sub('[ ]{2,}', ' ', text)

  # case normalize and strip extra white spaces on the far left and right hand side
  text = text.lower().lstrip().rstrip()
  return text

train['description'] = train['description'].apply(clean_doc)
test['description'] = test['description'].apply(clean_doc)


In [40]:
train['description'][0]

'a marriage of and year old bourbons a mature yet very elegant whiskey with a silky texture and so easy to embrace with a splash of water balanced notes of honeyed vanilla soft caramel a basket of complex orchard fruit blackberry papaya and a dusting of cocoa and nutmeg smooth finish sophisticated stylish with well defined flavors a classic'

### 1.2 Split training data into Feature Matrix `X` and Target Vector `y`

In [42]:
target = 'category'
# COMPLETE THE CODE IN THIS CELL
y = train[target]
X = train['description']
X_test = test['description']

In [48]:
import numpy as np
print(np.sort(np.unique(y)))
print(X.shape)
print(X_test.shape)
print(np.histogram(y,bins=[0.5, 1.5, 2.5, 3.5, 4.5])[0])

[1 2 3 4]
(2586,)
(288,)
[1637  449  300  200]


### 1.3 Specify the Model and Define the Pipeline Components

For the classifier model, you can try any or several of
* `RandomForestClassifier()` or `GradientBoostingClassifier()` from the `sklearn` library
* `XGBClassifier()` from the `xgboost` library
* `CatboostClassifier()` from the `catboost` library
* `LGBMClassifier()` from the `lightgbm` library


In [52]:
# limit max_features to 500 to speed up training on Colab.
# COMPLETE THE CODE IN THIS CELL

max_features=500

# Use tfdi rather than count vectorizer, ngram 1,2 includes bigram
vect = TfidfVectorizer(stop_words='english', max_features=max_features, ngram_range=(1,2))
# clf = XGBClassifier(random_state=42) That's what the assignment wanted me to use versus the solution video
                                      # However, it didn't work. It says my model could be misconfigured when I tried to fit the model
clf = LGBMClassifier(learning_rate=0.1, max_depth=-5, random_state=42) # what solution video used

# Give it a name as a string
pipe = Pipeline([('vect', vect), ('clf', clf)])

### 1.4 Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model.

In [53]:
# COMPLETE THE CODE IN THIS CELL
# Parameters to search in dictionary, has to have curly braces. These were what the instructor used in the solution video.
parameters = {
    'vect__max_features': ([500]),
    'vect__analyzer': (['word']),
    'vect__max_df': ([0.75]), # filter out most common words
    'clf__max_depth':([15]), # of the tree
    'clf__n_estimators': ([1500]), # max number of trees in the forest
    'clf__learning_rate': ([0.1]),
    'clf__num_leaves': ([50]), # leaves at the bottom of the decision tree
    'clf__min_data_in_leaf': ([30]) # min num of data points in each leaf
}

# Implement a grid search with cross-validation
grid_search = GridSearchCV(pipe,parameters,cv=3, n_jobs=-1, verbose=2) # This is just for cross-val because we only have 1 set of parameters
grid_search.fit(X, y)

# Display the best score from the grid search
grid_search.best_score_

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

0.8952049497293116

In [54]:
# Display the best parameters from the grid search
print(grid_search.best_params_)

{'clf__learning_rate': 0.1, 'clf__max_depth': 15, 'clf__min_data_in_leaf': 30, 'clf__n_estimators': 1500, 'clf__num_leaves': 50, 'vect__analyzer': 'word', 'vect__max_df': 0.75, 'vect__max_features': 500}


### 1.5 Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so only submit when your predicted test accuracy is the highest you can make it. For this competition the max daily submissions are capped at **20**.  The submission file is made from the results of running your best model on the **test data set**, for which we don't get the targets.<br><br>

In [55]:
# COMPLETE THE CODE IN THIS CELL
# Predictions on **test** sample
pred = grid_search.predict(test['description'])

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


In [56]:
# COMPLETE THE CODE IN THIS CELL
# Create pandas dataframe
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [57]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [58]:
submission.shape

(288, 2)

In [59]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission_number = 1 # submission_number = 0 was the original line in the assignment

submission.to_csv(f'submission{submission_number}.csv', index=False)
# submission_number += 1 in the original line in the assignment

In [60]:
# Download submission to local machine from this Google Colab notebook
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1.6 Submit your results to `kaggle` and get your score

First, upload the `kaggle.json` API token file from your local machine.<br>
Do this by clicking the file icon in the left sidebar, <br>
then clicking file icon with an up arrow inside it at the upper left, <br>
then navigating to and selecting the `kaggle.json` file in your local machine.<br>
`kaggle.json` is usually found in a folder called `.kaggle` on your local machine, <br>
if you have a Kaggle account. Note that there is no cost involved in <br>registering for a Kaggle account.<br><br>

Then: make a folder `/root/.kaggle` in this notebook,<br>
and copy your `kaggle.json` file into the `/root/.kaggle/` folder

In [29]:
#!mkdir /root/.kaggle/ # make a folder here
!mv /kaggle.json /root/.kaggle/ # move my kaggle.json file in my local machine to the folder here. My local file is stored in This PC > C: > Users > roadb > .kaggle
!chmod 600 /root/.kaggle/kaggle.json # change the kaggle file to safeguard your privacy so others can't see my credentials
!ls -l /root/.kaggle/

total 4
-rw------- 1 root root 68 May 11 01:25 kaggle.json


Now you can submit your predictions to Kaggle
directly from this Colab notebook!<br> If it says "You must accept rules..." then go to Kaggle website and click on the Submission button and the accept rules option will pop up. Then re-run this cell.


In [61]:
# Get the name of the competition on the partial URL of the webpage
!kaggle competitions submit whiskey-201911 -f submission1.csv -m "submission1"

100% 1.91k/1.91k [00:01<00:00, 1.59kB/s]
Successfully submitted to Whiskey

You can check your score on the [Kaggle Whisky Competition website](https://www.kaggle.com/c/whiskey-201911/submissions),
<br>on the "My Submissions" tab:

_This submission got a score of $0.94186$_

## Challenge

You're trying to achieve a minimum of 75% Accuracy on your model.

## 2. Add Latent Semantic Indexing to your pipeline (Learn)
<a id="p2"></a>

### Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try:
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (LSI) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle


### 2.1 Define Pipeline Components

Nest pipelines to perform SVD on our vectorization (LSA)

Everything is the same as Part 1 except now we're adding LSA to the pipeline.

In [63]:
# COMPLETE THE CODE IN THIS CELL
# Transforming our Vectorization with SVD is how LSA generates topic columns
svd = TruncatedSVD(n_components=2, # number of topics to generate (also the size of the new feature space)
                   algorithm='randomized',
                   n_iter=10)

# vectorizer and classifier like before
vect = TfidfVectorizer(stop_words='english', max_features=500)
# clf = XGBClassifier()
clf = LGBMClassifier() # what solution video used

# LSA pipeline with vectorizer & truncated SVD
lsa = Pipeline([('vect', vect), ('svd', svd)])

# combine LSA pipeline together with classifier
pipe = Pipeline([('lsa', lsa), ('clf', clf)])

### 2.2 Define Your grid search space and run a grid search with cross-validation
You're looking for both the best hyperparameters of your vectorizer and your classification model.

In [64]:
# COMPLETE THE CODE IN THIS CELL
parameters = {
    'lsa__svd__n_components': ([50]),
    'lsa__vect__max_df': ([0.75]),
    'clf__max_depth': ([15])
    # 'clf__n_estimators': ([1500]),   other things to tune for the model
    # 'clf_learning_rate': ([0.1]),
    # 'clf_num_leaves': ([50]),
    # 'clf_min_data_in_leaf': ([30])
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 2586, number of used features: 50
[LightGBM] [Info] Start training from score -0.457247
[LightGBM] [Info] Start training from score -1.750845
[LightGBM] [Info] Start training from score -2.154085
[LightGBM] [Info] Start training from score -2.559550


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lsa',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(max_features=500,
                                                                         stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_iter=10))])),
                                       ('clf', LGBMClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [15], 'lsa__svd__n_components': [50],
                         'lsa__vect__max_df': [0.75]},
             verbose=1)

In [65]:
grid_search.best_score_

0.9021655065738593

In [66]:
grid_search.best_params_

{'clf__max_depth': 15, 'lsa__svd__n_components': 50, 'lsa__vect__max_df': 0.75}

### 2.3 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

In [67]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [68]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [69]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [72]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number = 2

In [73]:
# Download submission to your local machine from this Colab notebook
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:
!kaggle competitions submit whiskey-201911 -f submission1.csv -m "submission2"

100% 1.91k/1.91k [00:01<00:00, 1.44kB/s]
Successfully submitted to Whiskey

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets.

# 3. Add Spacy Word Embeddings
<a id="p3"></a>

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try:
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use document vectors made from those word embeddings as your features for a classification model.
4. Make a submission to Kaggle

### 3.1 Process the data set with spacy
Instead of TFIDF vectorizer, let's use spacy word embeddings - producing a vector for each word. To get the vector for the document (a review), average all the word vectors. And each word vector has a 300 dimension. Bag of words model. We'll get a 300 dimensional vector for our document vector.

In [74]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {

    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [75]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_md")

def get_word_vectors(docs): # docs is a list of text strings
    # YOUR CODE HERE
    return [nlp(doc).vector for doc in docs]

X_train_emb = get_word_vectors(train['description'])
X_test_emb = get_word_vectors(test['description'])

In [78]:
len(X_train_emb[0])

300

In [76]:
rfc = RandomForestClassifier(oob_score=True)

rfc.fit(X_train_emb, y)

RandomForestClassifier(oob_score=True)

In [77]:
# massively overfit with the Random Forest
print('Training Accuracy: ', rfc.score(X_train_emb, y))

Training Accuracy:  1.0


Here we use oob_score_ (out-of-bag score) as a **proxy** for the test score;<br>
for your submission, you will predict on the test set, as before

In [79]:
# validation looks decent without any tuning

rfc.oob_score_

0.6805877803557618

### 3.2 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

### Make a Submission File

In [80]:
# YOUR CODE HERE

In [81]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission_number = 3
submission.to_csv(f'submission{submission_number}.csv', index=False)

In [82]:
# Download submission to local machine from Google Colab
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3.3 Submit your predictions to Kaggle


---



In [84]:
# YOUR CODE HERE
!kaggle competitions submit whiskey-201911 -f submission1.csv -m "submission3"

100% 1.91k/1.91k [00:00<00:00, 2.23kB/s]
Successfully submitted to Whiskey

# Post Lecture Assignment (Stretch)
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. <br>
Once you've accomplished that, do (1), and either (2) or (3):

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions:
    - What is "Sentiment Analysis"?
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do people create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?

2. Singular Value Decomposition (SVD) is one of the most important and powerful methods in Applied Mathematics and in all of Machine Learning.  Principal Components Analysis (PCA) -- which we used in Module 2 -- is closely releated to SVD. Research SVD using the resources below. Then write a few paragraphs explaining -- in your own words -- your understanding of SVD and why it has become so important in Machine Learning. As you write, pretend that you will be presenting this summary orally as an answer to a question during a job interview.<br>

* [Daniela Witten](https://www.danielawitten.com/), a Professor of Mathematical Statistics at the University of Washington, recently penned a highly amusing and informative [tweetstorm](https://twitter.com/WomenInStat/status/1285611042446413824) about SVD, well worth reading!<br>
* [Stanford University Lecture on SVD](https://www.youtube.com/watch?v=P5mlg91as1c) <br>
* [StatQuest Principal Components Analysis](https://www.youtube.com/watch?v=FgakZw6K1QQ)<br>
* [Luis Serrano Principal Components Analysis](https://www.youtube.com/watch?v=g-Hb26agBFg)<br>

3. Research which other models can be used for text classification -- see [Multi-Class Text Classification Model Comparison and Selection](https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568)
  - Try a few other classical machine learning models, and compare with the gradient boosting results
  - Neural Networks are becoming more popular for document classification. Why is that the case?
  - If you have the time and interest, check out this [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google
   